# Visualisation of OpenImpala Data

The aim of this notebook is to demonstrate how to visualise plots produced using OpenImpala. For this example all files are stored in the Git repository at: [github.com/jameslehoux/openimpala-jupyter](github.com/jameslehoux/openimpala-jupyter), and we will be using the open-source visualisation software, yt: [yt-project.org](yt-project.org)

## Test Data

We have provided a set of test data that will be used to demonstrate different visualisation functionality. These datasets are 3D image datasets of a lithium iron phosphate electrode:

* *tiffreadertest* is a Tiff image that has been obtained from X-Ray tomography data, tomography data is produced as greyscale values and it has been thresholded into 2 domains.

* *diffusionplot* is the same Tiff image from *tiffreadertest*, but a steady-state Fickian diffusion problem has been run in the x-direction across the domain.

## Visualisation of Tiff Data

In order to visualise the Tiff data, first we need to load yt, and then create a dataset, *'tiffds'*, that points to testdata directory. 

We can then probe the dataset object using *'field_list'* to see the properties associated with that dataset, like: voxel dimensions, the real physical space taken up by the domain and the parameter values associated with the 3D space (e.g. phase).

In [ ]:
import yt

tiffds = yt.load("testdata/tiffreadertest")

tiffds.field_list

We can then use this dataset, *'tiffds'*, and plot a 2D representation of the phase information. To do this we can use a *SlicePlot*, specify the plane we want to slice, and specify the parameter we want to plot. Next we annotate the plot and choose the colour map and plotting characteristics:

In [ ]:
slc = yt.SlicePlot(tiffds, "y", "phase")
slc.annotate_title('This is a plot to show the phase composition')

slc.set_log("phase", False)

slc.set_cmap("phase", "B-W LINEAR")

slc.show()


*Now try modifying the code above to produce a different plot, this time in the x-plane.* 

## Diffusion Problem

The diffusion field is calculated through using Fick's law:

$$
J_d = -D \nabla \varphi
$$
where $D$, the diffusion coefficient, is:

$$
D = 1\,\text{for the "white" phase}
$$

$$
D = 0\,\text{for the "black" phase}
$$

so there is only conductivity in one phase in this case, and $\nabla \varphi$, the concentration gradient, is calculated using: 

$$
\begin{cases}
\nabla ^2\varphi = 0, & \text{in} \quad \Omega,\\
\varphi = 1, & \text{on} \quad T,\\
\nabla \varphi \cdot \mathrm{\textbf{n}}  = 0, & \text{on} \quad I,\\
\varphi = -1, & \text{on} \quad B\\
\end{cases}
$$

where $\varphi$ is the local concentration of the diffusing species, $\Omega$ is the conductive region of the porous medium, $\mathrm{\textbf{n}}$ is the outward pointing unit normal to $\Omega$ and $T$, $B$ and $I$ are the top, bottom and interfacial faces respectively.

## 2D Visualisation of Diffusion Data

To visualise the diffusion data, we need to again load the data into a dataset object, this time, *'ds'*, and then we can probe the dataset properties.

*How does the dataset differ to the 'tiffds' example?*

In [ ]:
import yt

ds = yt.load("testdata/diffusionplot")

ds.field_list

We can then use the same commands from the *'tiffds'* example to plot the phase information:

In [ ]:
slc = yt.SlicePlot(ds, "y", "phase")
slc.annotate_title('This is a phase plot')

slc.set_log("phase", False)

slc.set_cmap("phase", "B-W LINEAR")

slc.show()

But as we can see from the field_list probe, this dataset also contains values for *concentration*. We can change the properties of the SlicePlot to display this data instead:

In [ ]:
slc = yt.SlicePlot(ds, "y", "concentration")
slc.annotate_title('This is a concentration plot')
slc.set_log("concentration", False)

slc.set_cmap("concentration", "kamae")

slc.show()


*Now try changing the colourmap of the plot above, you can use any matplotlib colourmaps or from this list: [yt-project.org/docs/dev/visualizing/colormaps/index.html](https://yt-project.org/docs/dev/visualizing/colormaps/index.html)*



We can also plot projection plots, where the full depth of values into the plane is summed. These can help to build up understanding of constrictions and openings in structures, and how they affect flowpaths:

In [ ]:
slc = yt.ProjectionPlot(ds, "y", ["concentration"])
slc.annotate_title('This is a projection plot')
slc.set_log("concentration", False)

slc.set_cmap("concentration", "hot")

slc.show()

slc.set_cmap("concentration", "arbre")

slc.show()

We can also weight these projection plots to remove the influence of the phase value:

In [ ]:

slc = yt.ProjectionPlot(ds, "y", "concentration", weight_field="phase")
slc.annotate_title('This is a weighted projection plot')
slc.set_log("concentration", False)

slc.set_cmap("concentration", "arbre")

    


slc.show()

## 3D Visualisation of Diffusion Data

To visualise the diffusion data in 3D, we need to create something called a *scene*. A *scene* contains camera, lighting and dataset information, and tells yt what properties are important to be visualised.

For example, if we wanted to plot a 3D representation of the phase field, we could use:

In [ ]:
sc = yt.create_scene(ds, field="phase")
source = sc[0]
source.tfh.set_bounds((0.000000000001, 1))
source.tfh.set_log(True)
source.tfh.grey_opacity = True
sc.camera.resolution = 1024
sc.camera.switch_orientation()
sc.annotate_axes(alpha=.5)
sc.annotate_domain(ds, color=[1, 1, 1, 0.1])
sc.show(sigma_clip = 3)

Or if we wanted to plot a 3D represenatation of the concentration field, we could use:

In [ ]:
sc = yt.create_scene(ds, field="concentration")


source = sc[0]

source.set_field('concentration')
source.set_log(False)
source.tfh.grey_opacity = True

bounds = (1 ,2)
sc.camera.focus = ds.domain_center
sc.camera.resolution = 1024
sc.camera.north_vector = [0, -1, 0]
sc.camera.position = [-1, -1, -1]
slc.set_log("concentration", False)
slc.set_log("phase", False)

ax = slc.plots['concentration'].axes

# Since this rendering is done in log space, the transfer function needs
# to be specified in log space.
tf= yt.ColorTransferFunction(bounds)

render_source = sc.get_source()
render_source.transfer_function.clear()
render_source.transfer_function.map_to_colormap(
    (ds.quan(1.0)),
    (ds.quan(2.0)),
    scale=5.0, colormap='arbre')
#tf.add_layers(2, colormap='arbre')

#source.tfh.tf = tf
#source.tfh.bounds = bounds
sc.annotate_axes(alpha=.5)
sc.annotate_domain(ds, color=[1, 1, 1, 0.1])

sc.show(sigma_clip = 3)


These 3D *scenes* are noticeably more complex than the 2D equivalents, and take more computing power to perform the visualisation. They are, however, useful tools in understanding the 3-dimensional influence of structure on flowpaths. To explore these ideas further, it is recommended to look at the yt wiki, available: [https://yt-project.org/docs/dev/visualizing/volume_rendering.html](https://yt-project.org/docs/dev/visualizing/volume_rendering.html) 